In [1]:
%pip install python-telegram-bot

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install nest_asyncio

In [ ]:
import logging
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import nest_asyncio

# Applying nest_asyncio to allow multiple event loops
nest_asyncio.apply()

# Logging setup
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Loading TinyLlama model and tokenizer
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
print("Model loaded successfully!")

# Function to generate a response from TinyLlama
def generate_response(prompt: str) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# /start command handler
async def start(update: Update, context: CallbackContext) -> None:
    user = update.effective_user
    await update.message.reply_text(f"Hello {user.first_name}, I'm your AI assistant powered by TinyLlama!")

# Message processing handler
async def process(update: Update, context: CallbackContext) -> None:
    user_message = update.message.text
    await update.message.reply_text("Processing your message...")
    try:
        # Generate response from TinyLlama
        ai_response = generate_response(user_message)
        await update.message.reply_text(ai_response)
    except Exception as e:
        logger.error(f"Error generating response: {e}")
        await update.message.reply_text("Sorry, I encountered an error while processing your request.")

# Main function to start the bot
def main() -> None:
    """Start the Telegram bot."""
    API_TOKEN = "7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs"

    # Initialize the application
    application = Application.builder().token(API_TOKEN).build()

    # Add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, process))

    # Run the bot
    application.run_polling()

# Entry point
if __name__ == '__main__':
    main()

Loading model...
Model loaded successfully!


2025-01-25 17:36:38,168 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs/getMe "HTTP/1.1 200 OK"
2025-01-25 17:36:39,047 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs/deleteWebhook "HTTP/1.1 200 OK"
2025-01-25 17:36:39,047 - telegram.ext.Application - INFO - Application started
2025-01-25 17:36:45,899 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs/getUpdates "HTTP/1.1 200 OK"
2025-01-25 17:36:46,330 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs/sendMessage "HTTP/1.1 200 OK"
2025-01-25 17:36:54,846 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AAGqvH0wEafzPuMlg_Hh5wh9lkZxj5IA4zs/getUpdates "HTTP/1.1 200 OK"
2025-01-25 17:36:55,823 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7901574935:AA

RuntimeError: Cannot close a running event loop